# Evaluations with Langfuse

In [ ]:
! pip install langsmith openai langfuse pandas
! pip install -qU requests bs4 lxml chromadb langchain langchain-text-splitters langchain-openai
! pip install -qU duckduckgo-search langchain-community ddgs
! pip install unstructured jq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.6/364.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 2.0.0
    Uninstalling wrapt-2.0.0:
      Successfully uninstalled wrapt-2.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 14.1 MB/s eta 0:00:00
   ━━━━━

In [ ]:
import os

os.environ["LANGFUSE_TRACING"] = "true"
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"
os.environ["LANGFUSE_SECRET_KEY"] = ""
os.environ["LANGFUSE_PUBLIC_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
# kb_en_to_chroma.py  — minimal & direct
import os, re, time, requests
from urllib.parse import urljoin, urldefrag
from bs4 import BeautifulSoup

BASE = "https://www.kapitalbank.az"
START = f"{BASE}/en"
UA = {"User-Agent": "kb-minicrawl/0.2"}
TIMEOUT = 15
MAX_PAGES = 50

def clean_url(u):
    u = urldefrag(u)[0]
    if not u: return None
    if not u.startswith("http"): u = urljoin(BASE, u)
    if not u.startswith(START): return None
    if re.search(r"\.(pdf|jpe?g|png|gif|svg|mp4|zip|docx?|xlsx?)$", u, re.I): return None
    return u

def extract_text(html):
    s = BeautifulSoup(html, "lxml")
    for t in s(["script","style","noscript","svg","footer","nav","header"]): t.decompose()
    n = s.select_one("main") or s.select_one("article") or s.body or s
    return " ".join((n.get_text(" ", strip=True) if n else s.get_text(" ", strip=True)).split())

visited, queue, pages = set(), [START], []
while queue and len(visited) < MAX_PAGES:
    url = queue.pop(0)
    if url in visited: continue
    try:
        r = requests.get(url, headers=UA, timeout=TIMEOUT)
        if r.ok and "text/html" in r.headers.get("Content-Type",""):
            txt = extract_text(r.text)
            if len(txt) > 200:
                pages.append({"url": url, "text": txt})
            s = BeautifulSoup(r.text, "lxml")
            for a in s.find_all("a", href=True):
                u = clean_url(a["href"])
                if u and u not in visited:
                    queue.append(u)
        visited.add(url); time.sleep(0.15)
    except requests.RequestException:
        visited.add(url)

import json

# Save the crawled pages data to a file for later use
pages_outfile = "kapitalbank_pages.json"
with open(pages_outfile, "w", encoding="utf-8") as f:
    json.dump(pages, f, indent=2, ensure_ascii=False)
print(f"Saved {len(pages)} pages to {pages_outfile}")

# Load crawled pages from JSON file to make them available for Chroma processing
with open(pages_outfile, "r", encoding="utf-8") as f:
    pages = json.load(f)
print(f"Loaded {len(pages)} pages from {pages_outfile}")

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# ---- LangChain chunking ----
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=120)
docs, metas = [], []
for p in pages:
    for chunk in splitter.split_text(p["text"]):
        docs.append(chunk)
        metas.append({"url": p["url"]})

# ---- OpenAI embeddings -> Chroma ----
persist_dir = "chroma_kapitalbank"
emb = OpenAIEmbeddings(model="text-embedding-3-small")  # cheap & solid
vs = Chroma.from_texts(
    texts=docs,
    embedding=emb,
    persist_directory=persist_dir,
    collection_name="kapitalbank_en",
    metadatas=metas,
)
vs.persist()
print(f"Indexed pages={len(pages)} chunks={len(docs)} into {persist_dir}/ (collection 'kapitalbank_en')")

from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

persist_dir = "chroma_kapitalbank"
collection_name = "kapitalbank_en"
emb = OpenAIEmbeddings(model="text-embedding-3-small")

# Load the existing/persisted Chroma vector store
vs = Chroma(
    persist_directory=persist_dir,
    embedding_function=emb,
    collection_name=collection_name
)

Saved 39 pages to kapitalbank_pages.json
Loaded 39 pages from kapitalbank_pages.json


/tmp/ipython-input-2284835735.py:80: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vs.persist()


Indexed pages=39 chunks=161 into chroma_kapitalbank/ (collection 'kapitalbank_en')


/tmp/ipython-input-2284835735.py:91: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vs = Chroma(


## Run our agent on a dataset

In [ ]:
# pip install -U "langchain>=0.3" "langgraph>=0.2" "langchain-openai>=1.0.0" "langfuse>=2.0.0"

from typing import Any, Dict
from langchain.chat_models import init_chat_model
from langchain_core.tools import create_retriever_tool, tool, render_text_description
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import create_agent

from langfuse import get_client
from langfuse.langchain import CallbackHandler
from langfuse import observe

from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

persist_dir = "chroma_kapitalbank"
collection_name = "kapitalbank_en"
emb = OpenAIEmbeddings(model="text-embedding-3-small")

# Load the existing/persisted Chroma vector store
vs = Chroma(
    persist_directory=persist_dir,
    embedding_function=emb,
    collection_name=collection_name
)

# --- Langfuse: prompt mgmt + tracing ---
langfuse = get_client()
lf_cb = CallbackHandler()

# Get current production version (with a safe fallback for HA)
lf_prompt = langfuse.get_prompt(
    "general-agent-prompt",
    label="production",
    type="text",
    fallback="Answer the user's question. Prefer internal knowledge before the web.\n\nTools:\n{{tools}}"
)

# --- Tools (assume your vector store exists as `vs`) ---
retriever = vs.as_retriever(search_kwargs={"k": 3})
retrieve_tool = create_retriever_tool(
    retriever=retriever,
    name="retrieve",
    description="Search the internal vector store for passages relevant to the user's question."
)

_ddg = DuckDuckGoSearchRun()
@tool("duckduckgo_search")
def duckduckgo_search(query: str) -> str:
    """Search the web with DuckDuckGo and return a brief summary of top results."""
    return _ddg.run(query)

TOOLS = [retrieve_tool, duckduckgo_search]

# --- Model: respect prompt.config if set; otherwise default ---
cfg = lf_prompt.config or {}
model_id = cfg.get("model", "openai:gpt-4o-mini")
temperature = float(cfg.get("temperature", 0))
model = init_chat_model(model_id, temperature=temperature)

# --- Build system prompt from Langfuse prompt ---
tools_desc = render_text_description(TOOLS)
if "{{tools}}" in (lf_prompt.prompt or ""):
    system_prompt = lf_prompt.compile(tools=tools_desc)  # render {{tools}}
else:
    # If your prompt has no placeholder, just append a tool section.
    system_prompt = (lf_prompt.prompt or "") + f"\n\n# Tools\n{tools_desc}"

# --- Agent ---
@observe()  # traces the function; pair with the callback on invoke
def agentic_solution(inputs: Dict[str, Any], **kwargs) -> Dict[str, Any]:
    q: str = inputs["question"]

    agent = create_agent(
        model=model,
        tools=TOOLS,
        system_prompt=system_prompt,  # accepts a plain string
    )

    res = agent.invoke(
        {"messages": [{"role": "user", "content": q}]},
        config={"callbacks": [lf_cb]},  # Langfuse tracing
    )

    # Robustly extract final text (dict w/ "messages" or AIMessage)
    msg = res["messages"][-1].content if isinstance(res, dict) and "messages" in res else getattr(res, "content", str(res))
    return {"answer": msg.strip()}


In [ ]:
from langfuse import get_client

# Initialize client
langfuse = get_client()

def agent_task(*, item, **kwargs):
    question = getattr(item, "input", None) if not isinstance(item, dict) else item.get("input")
    if not isinstance(question, str):
        raise ValueError(f"Bad item; no 'input' found. Got: {type(item)}")
    out = agentic_solution(inputs={"question": question})
    return out.get("answer") if isinstance(out, dict) else str(out)

# Get dataset from Langfuse
dataset = langfuse.get_dataset("deepeval_synthetic_data") # openai_synthetic_dataset / deepeval_synthetic_data

# Run experiment directly on the dataset
result = dataset.run_experiment(
    name="Synthetic Dataset Test",
    description="Synthetic dataset test",
    task=agent_task # see above for the task definition
)

# Use format method to display results
print(result.format())

Individual Results: Hidden (12 items)\n💡 Set include_item_results=True to view them\n\n──────────────────────────────────────────────────\n🧪 Experiment: Synthetic Dataset Test
📋 Run name: Synthetic Dataset Test - 2025-11-01T15:09:43.834561Z - Synthetic dataset test\n12 items\n🔗 Dataset Run:\n   https://cloud.langfuse.com/project/cmgo79g7x070uad07qc7g98st/datasets/cmh4fxiog04cuad07miowcuad/runs/f6ae03ff-0802-40ad-8c5c-25d6be1d338f


In [ ]:
# Use format method to display results
print(result.format(include_item_results=True))

\n1. Item 1:\n   Input:    How do integrity, collaboration, and product quali...\n   Expected: Integrity, collaboration, and product quality are ...\n   Actual:   Kapital Bank's leadership in the retail market and...\n\n   Trace ID: a36372199f3b0b4621e42a47c444020a\n\n2. Item 2:\n   Input:    How do Kapital Bank’s values of integrity, collabo...\n   Expected: Kapital Bank’s values of integrity, collaboration,...\n   Actual:   Kapital Bank emphasizes three core values: integri...\n\n   Trace ID: 70f20d57e00269b855eec9aff39f3fef\n\n3. Item 3:\n   Input:    How do Kapital Bank deposit safes ensure security,...\n   Expected: Kapital Bank deposit safes ensure security by usin...\n   Actual:   Kapital Bank deposit safes provide a secure and co...\n\n   Trace ID: b3a92e3c92d6d403745de14029d53c57\n\n4. Item 4:\n   Input:    How do termless deposits and high-interest savings...\n   Expected: Termless deposits have no fixed duration and allow...\n   Actual:   Termless deposits and high-interest 